In [1]:
import os
import string
from nltk.corpus import stopwords
import nltk
import huspacy
import re

## 3.1. Data Preprocessing

Reading in the files as list of strings with each element of the list being a document

In [2]:
# old documents (before the 16th century)
old_files = [f for f in os.listdir('../data/old') if f.endswith('.txt')]
old_documents = []
for filename in old_files:
    with open(os.path.join('../data/old',filename),'r',encoding='utf-8') as f:
        old_documents.append(f.read())

# new documents (from 20th century)
new_files = [f for f in os.listdir('../data/new') if f.endswith('.txt')]
new_documents = []
for filename in new_files:
    with open(os.path.join('../data/new',filename),'r',encoding='utf-8') as f:
        new_documents.append(f.read())

In [3]:
# inspecting the results of reading in the files
print("An example document in old Hungarian:")
print(old_documents[8])
print("------------------------------------------")
print("An example document from the 20th century:")
print(new_documents[10])

An example document in old Hungarian:
Krisztus feltámada , mint ön nagy kínjából asszony , mi is örülünk .
Krisztus legyen reményünk .
Kyrie eleison 
------------------------------------------
An example document from the 20th century:
Nincsen apám, se anyám,
se istenem, se hazám,
se bölcsőm, se szemfedőm,
se csókom, se szeretőm.

Harmadnapja nem eszek,
se sokat, se keveset.
Húsz esztendőm hatalom,
húsz esztendőm eladom.

Hogyha nem kell senkinek,
hát az ördög veszi meg.
Tiszta szívvel betörök,
ha kell, embert is ölök.

Elfognak és felkötnek,
áldott földdel elfödnek
s halált hozó fű terem
gyönyörűszép szívemen.


In [4]:
# download time: 59.3s
nltk.download("stopwords")
huspacy.download()

[nltk_data] Downloading package stopwords to C:\Users\Kim
[nltk_data]     Levente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['c:\\ProgramData\\anaconda3\\python.exe', '-m', 'pip', 'install', 'hu_core_news_lg @ https://huggingface.co/huspacy/hu_core_news_lg/resolve/v3.8.0/hu_core_news_lg-any-py3-none-any.whl']


In [5]:
def TextPreprocessor(text, nlp):
    """ Preprocesses documents

    Parameters
    ----------
    text string, document to be cleaned
    nlp Spacy Language, pipeline for tokenization
    
    Returns
    -------
    text_clean string, cleaned document
    """
    # make text lowercase, replace linebreak with space
    text_low = text.lower()
    text_low = text_low.replace("\n"," ")

    # some texts contain indicators of which line the text is at
    # in curly brackets
    text_low = re.sub(r'\{[^}]*\}', '', text_low)

    # tokenize the document
    tokens = [token.text for token in nlp(text_low)]
    
    # exclude stopwords and punctuations
    stop_words = set(stopwords.words("Hungarian"))
    tokens_clean = [word for word in tokens if word not in stop_words and word not in string.punctuation]

    # cleaned tokens are joined by space
    text_clean = " ".join(tokens_clean)

    return text_clean

In [6]:
# clean texts, runtime: 19.4s
nlp_hun = huspacy.load()
old_docs_clean = [TextPreprocessor(doc,nlp_hun) for doc in old_documents]
new_docs_clean = [TextPreprocessor(doc,nlp_hun) for doc in new_documents]

Inspecting the result of preprocessing an old text and a new one

In [7]:
print(old_documents[4])
print("--------------")
print(old_docs_clean[4])

ez vég pusztaságról megemlékeznétek , el ne vesznétek , az régi jó nevet megelevenítenétek , kereszténységnek jó vértei  lennétek .
szegény Mátyás király vala békességben , mert országa vala egyességben , vitézek valának nála tisztességben , az urak valának nagy egyenességben .
ti Úristen ellen ne háborganátok , régi dekrétumot csak megtartanátok , az dézsmát igazán kiszolgáltatnátok , koronként Istennek vele áldoznátok .
Isten , Szűz Mária háborútól védjen , Hatvanban gyűlétek , hogy jó végre legyen , jó Lajos királyunk diadalmat vegyen , minden tanácsotokban ő jó véget tegyen .
Pesti Beke Ferenc szíve kétségében , ki az vendég népnek bízik erejében , török császárt töri hízelkedésében , minden ennek ő elvetett beszédében .
Geszti László diák szerzé ez éneket , Magyarország vala nagy fő szükségében , az végek valának mind elveszendőben , ezerötszázhuszonöt esztendőben .
--------------
vég pusztaságról megemlékeznétek vesznétek régi nevet megelevenítenétek kereszténységnek vértei   len

In [12]:
print(new_documents[4])
print("--------------")
print(new_docs_clean[4])

Haragszom én arra szóra,
Ki a papot úgy megszólja,
Mert a papnak nincs bundája,
Hideg a reverendája.
Heje-huja, hopp!
Haragszom én arra szóra,
Ki a mestert úgy megszólja;
A mesternek nincs kalapja,
Sapkában jár az utcára.
Heje-huja, hopp!
Heje-huja, szűröm ujja!
A cigány a nótám fújja.
Gyere, rózsám, táncoljunk hát,
Járjuk el a magyar nótát!
Heje-huja, hopp!
--------------
haragszom szóra papot megszólja papnak bundája hideg reverendája heje-huja hopp haragszom szóra mestert megszólja mesternek kalapja sapkában jár utcára heje-huja hopp heje-huja szűröm ujja cigány nótám fújja gyere rózsám táncoljunk hát járjuk magyar nótát heje-huja hopp


In [11]:
# get example stop words in Hungarian
stopwords.words("Hungarian")[0:20]

['a',
 'ahogy',
 'ahol',
 'aki',
 'akik',
 'akkor',
 'alatt',
 'által',
 'általában',
 'amely',
 'amelyek',
 'amelyekben',
 'amelyeket',
 'amelyet',
 'amelynek',
 'ami',
 'amit',
 'amolyan',
 'amíg',
 'amikor']

## 3.2. Embedding Models

## 3.3. Description of Classification Models and Results